<a href="https://colab.research.google.com/github/adolfoguimaraes/pyne_palestra_twitter/blob/main/code/coleta_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coletando dados do twitter com Python

Esse material foi criado para servir de suporte para a palestra **Coletando dados do Twitter com Python** que foi apresentada na Python Nordeste 2022 em Aracaju/SE. 

## Conteúdo

* Slides da Apresentação
* A biblioteca Twarc
* Métodos básicos de consulta

### Slides da Apresentação 

<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vSvNLeL_3Yi3Nk0kRE2_VmGjzGV6b778xE_lu6LhVPZXJCG6P9bPHEiC00r9ZYGz898lVugXMg1JdwJ/embed?start=false&loop=false&delayms=3000" frameborder="0" width="480" height="299" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

### A biblioteca Twarc

Esse repositório possui scripts em python para realizar a coleta de tweets utilizando a biblioteca [TWARC](https://twarc-project.readthedocs.io/en/latest/). A biblioteca permite utilizar a versão mais nova da API do Twitter (v2), além de ter acesso aos recursos para quem tem o _Academic Access Research_. A principal escolha dessa biblioteca foi por conta da possibilidade de utilizar o acesso acadêmico, quando o usuário tem essa permissão. 

A instalação pode ser feita a partir do comando `pip install twarc` e o seu uso pode ser feito a partir da linha de comando com a chamada do comando `twarc` ou por meio de scripts em python. Esse material usa a segunda abordagem.

A biblioteca permite acesso a diferentes métodos disponibilizados pela API do twitter. Vamos trabalhar as seguintes opções: 

* Busca dos tweets recentes a partir de uma string de busca. 
* Coleta de tweets a partir do métodos de `stream` (busca em tempo real).
* Coleta de tweets postados por um usuário.



In [ ]:
# Instalação das bibliotecas (apenas para quem for rodar direto no colab)

!pip install twarc 
!pip install jsonlines

In [ ]:
# imports necessários 

from twarc.expansions import flatten
from twarc.client2 import Twarc2
import jsonlines
import threading
import datetime
import pytz

In [ ]:
# Método auxiliar para impressão dos tweets. 
def print_basic_information(tweet):
    print("Criado em: %s" % tweet['created_at'])
    print("Criado por: %s" % tweet['author']['username'])
    print("Texto: %s " % tweet['text'])
    if 'referenced_tweets' in tweet.keys():
        print("Tweets referenciados:")
        for ref in tweet['referenced_tweets']:
            print("Tipo: %s" % ref['type'])
            print("Texto: %s " % ref['text'])
            print("-------")

### Acesso à API do Twitter

Para executar os scripts é necessário ter uma conta de desenvolvedor no Twitter. O cadastro de desenvolvedor pode ser feito no link: https://developer.twitter.com/. A chave utilizada vai ser o `Bearer Token` que pode ser gerado diretamente no portal de desenvolvedor. Essa chave é necessária para conectar à versão 2.0 da API do Twitter.

Alguns acessos só são permitidos para os usuários que têm o _Academic Research Access_. Mais detalhes dos requisitos para solicitar esse tipo de acesso estão disponíveis no link: https://developer.twitter.com/en/products/twitter-api/academic-research. 

### Credencias e instância da API

In [ ]:
# Credenciais 

BEARER_TOKEN = ""

In [ ]:
# Instanciando a biblioteca 

api_ = Twarc2(bearer_token=BEARER_TOKEN)

### Método `Search`

A busca dos tweets recentes podem ser feitas a partir do método `search_recent` que recebe os seguintes parâmetros:

In [ ]:
# Retorna um conjunto de páginas contendo `max_results` por página.
# Existe um limitação a depender do tipo de conta que esteja utilizando 
# Mas há uma limitação do próprio método que limita há um período de 7 dias.

response = api_.search_recent("#python", sort_order='recency', max_results=100)

limit_pages = 2
count_pages = 1
all_tweets = []
for page in response:
    tweets = flatten(page)
    print("Tweets coletados na página %i: %i" % (count_pages, len(tweets)))
    all_tweets.extend(tweets)
    if count_pages == limit_pages:
        break 

    count_pages += 1

print("Total coletado: %i" % len(all_tweets))


In [ ]:
# Imprimindo algums informações dos tweets coletados
print_basic_information(all_tweets[0])

**Salvando as informações em um arquivo:**

Ao invés de imprimir o resultado da busca na tela, vamos salvar em um arquivo `.jsonl`. Esse arquivo aceita o mesmo formato do json, com a diferença que cada linha armazena um arquivo json distinto. 

In [ ]:
with jsonlines.open("../output/arquivoteste.jsonl", mode="a") as writer:
    writer.write_all(all_tweets)

## Buscando em um `timeline`

In [ ]:
response_timeline = api_.timeline(user="adolfoguimaraes", exclude_retweets=True, exclude_replies=True, max_results=10)

limit_pages = 2
count_pages = 1
all_tweets_timeline = []
for page in response_timeline:
    tweets = flatten(page)
    print("Tweets coletados na página %i: %i" % (count_pages, len(tweets)))
    all_tweets_timeline.extend(tweets)
    if count_pages == limit_pages:
        break 

    count_pages += 1

print("Total coletado: %i" % len(all_tweets_timeline))


**Salvando dados em um arquivo:**

In [ ]:
with jsonlines.open("../output/arquivoteste2.jsonl", mode="a") as writer:
    writer.write_all(all_tweets_timeline)

## Busca em tempo real (`Stream`)

In [ ]:
# Regras de busca
new_rules = [{"value": "#pythonnordeste", "tag": "pythonnordeste"}]
rules_ = api_.add_stream_rules(new_rules)

In [ ]:
# Coletando tweets
event = threading.Event()

limit_ = 5

for count, result in enumerate(api_.stream(event=event)):
    tweet_ = flatten(result)
    for t in tweet_:
        print_basic_information(t)

    if count >= limit_ - 1:
        event.set()

In [ ]:
rule_ids = [r["id"] for r in rules_["data"]]
api_.delete_stream_rule_ids(rule_ids)

In [ ]:
api_.get_stream_rules()

## Busca `Archive`

A busca no arquivo do twitter está disponível apenas para quem tem a conta com acesso acadêmico. Ela permite realizar a busca dentro de um período estabelecido desde de 2006. No entanto, esse tipo de busca tem que ser feita com um pouco mais de cuidado porque o número de tweets retornados pode ser muito alto. 

In [ ]:
api_academic = Twarc2(bearer_token="")

In [ ]:
start_ = datetime.datetime.strptime("2022-08-25 00:00:00", "%Y-%m-%d %H:%M:%S").astimezone(tz=pytz.UTC)
end_ = datetime.datetime.strptime("2022-08-25 22:00:00", "%Y-%m-%d %H:%M:%S").astimezone(tz=pytz.UTC)

In [ ]:
all_tweets = []
for response_page in api_academic.search_all("#CartaPorUmGovernoEstadualAberto", sort_order='recency', max_results=100, start_time=start_, end_time=end_):
    tweets = flatten(response_page)
    print("Tweets coletados: %i " % len(tweets) )
    all_tweets.extend(tweets)

In [ ]:
with jsonlines.open("../output/arquivoteste3.jsonl", mode="a") as writer:
    writer.write_all(all_tweets)

É possível contar quantos tweets serão retornados a partir de uma busca. Isso pode ser bastante útil quando queremos diemensionar o tamanho de uma coleta. Vamos utilizar o método `counts`.

In [ ]:
start_ = datetime.datetime.strptime("2022-08-25 00:00:00", "%Y-%m-%d %H:%M:%S").astimezone(tz=pytz.UTC)
end_ = datetime.datetime.strptime("2022-08-25 22:00:00", "%Y-%m-%d %H:%M:%S").astimezone(tz=pytz.UTC)

In [ ]:
for response in api_academic.counts_all("#CartaPorUmGovernoEstadualAberto",start_time=start_, end_time=end_, granularity="minute"):
    counts = response['data']

In [ ]:
total_ = 0
for data_ in counts:
    total_minute = data_['tweet_count']
    hour_start = datetime.datetime.strptime(data_['start'], "%Y-%m-%dT%H:%M:%S.000Z").time()
    hour_end = datetime.datetime.strptime(data_['end'], "%Y-%m-%dT%H:%M:%S.000Z").time()
    total_ += total_minute

    print("De %s a %s: %i" % (hour_start, hour_end, total_minute))
    
    

print(total_)

## Busca com `Tweet Lookup`

Esse método permite coletar tweets a partir de um lista de IDs.



In [ ]:
all_tweets = []
for response in api_.tweet_lookup(tweet_ids=['1560706474556391428','1560275194131779586','1559904151588454403']):
    tweets = flatten(response)
    all_tweets.extend(tweets)
    

In [ ]:
print_basic_information(all_tweets[0])

# É isso 😊

Nesse material foi possível mostrar algumas das principais funções da biblioteca `twarc` para coleta de dados do twitter. É importante conhecer não só biblioteca mas todas as regras de uso desse tipo de API para fazer coletas de forma adequada. 

Obrigado e até a próxima. 